In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/Anxiety_Data.zip'
extract_path = '/content/Anxiety_Data'

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete. Contents:")
for root, dirs, files in os.walk(extract_path):
    for file in files:
        print(os.path.join(root, file))


Extraction complete. Contents:
/content/Anxiety_Data/anxiety_test.csv
/content/Anxiety_Data/anxiety_train.csv
/content/Anxiety_Data/anxiety_train_image/TR-991.jpg
/content/Anxiety_Data/anxiety_train_image/TR-1513.jpg
/content/Anxiety_Data/anxiety_train_image/TR-572.jpg
/content/Anxiety_Data/anxiety_train_image/TR-1450.jpg
/content/Anxiety_Data/anxiety_train_image/TR-1023.jpg
/content/Anxiety_Data/anxiety_train_image/TR-1269.jpg
/content/Anxiety_Data/anxiety_train_image/TR-302.jpg
/content/Anxiety_Data/anxiety_train_image/TR-2083.jpg
/content/Anxiety_Data/anxiety_train_image/TR-326.jpg
/content/Anxiety_Data/anxiety_train_image/TR-2553.jpg
/content/Anxiety_Data/anxiety_train_image/TR-1147.jpg
/content/Anxiety_Data/anxiety_train_image/TR-1288.jpg
/content/Anxiety_Data/anxiety_train_image/TR-1013.jpg
/content/Anxiety_Data/anxiety_train_image/TR-2423.jpg
/content/Anxiety_Data/anxiety_train_image/TR-1073.jpg
/content/Anxiety_Data/anxiety_train_image/TR-1332.jpg
/content/Anxiety_Data/anxiety_

In [ ]:
import os

image_dir = "/content/Anxiety_Data/anxiety_train_image"
image_count = len([f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff'))])

print(f"Number of Train images: {image_count}")


Number of Train images: 2608


In [ ]:
import os

image_dir = "/content/Anxiety_Data/anxiety_test_image"
image_count = len([f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff'))])

print(f"Number of Test images: {image_count}")


Number of Test images: 652


In [1]:
pip install  pytesseract


In [2]:
!apt-get update
!apt-get install -y tesseract-ocr

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,237 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,698 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/multiver

In [ ]:
import os
import pandas as pd
from PIL import Image
import pytesseract
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms


class AnxietyDataset(Dataset):
    def __init__(self, csv_file, image_folder, transform=None):
        self.image_folder = image_folder
        self.transform = transform

        data = pd.read_csv(csv_file)
        print("CSV columns:", data.columns)

        print("First few sample_ids from CSV:")
        print(data['sample_id'].head())

        data['filename'] = data['sample_id'].apply(lambda x: str(x) + ".jpg" if not str(x).endswith(".jpg") else x)

        data['exists'] = data['filename'].apply(lambda x: os.path.exists(os.path.join(self.image_folder, x)))
        print("Number of existing files found:", data['exists'].sum())

        self.data = data[data['exists']].reset_index(drop=True)
        print(f"Filtered dataset length: {len(self.data)} (only rows with existing images)")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image_name = row['filename']
        label = row['meme_anxiety_categories']
        image_path = os.path.join(self.image_folder, image_name)

        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        ocr_text = pytesseract.image_to_string(Image.open(image_path).convert("RGB"))

        return ocr_text, label

class FigurativeReasoningModule(nn.Module):
    def __init__(self):
        super(FigurativeReasoningModule, self).__init__()

    def forward(self, ocr_text):
        reasoning = "dummy figurative reasoning for: " + ocr_text
        return reasoning

if __name__ == "__main__":
    csv_train = '/content/Anxiety_Data/anxiety_train.csv'
    image_folder = '/content/Anxiety_Data/anxiety_train_image'

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    dataset = AnxietyDataset(csv_file=csv_train, image_folder=image_folder, transform=transform)
    dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

    figurative_module = FigurativeReasoningModule()
    
    for batch in dataloader:
        ocr_texts, labels = batch
        for text in ocr_texts:
            reasoning = figurative_module(text)
            print("OCR Text:", text)
            print("Figurative Reasoning:", reasoning)
        break


CSV columns: Index(['sample_id', 'meme_anxiety_categories'], dtype='object')
First few sample_ids from CSV:
0     TR-318
1    TR-1732
2    TR-1417
3    TR-1582
4     TR-649
Name: sample_id, dtype: object
Number of existing files found: 2605
Filtered dataset length: 2605 (only rows with existing images)
OCR Text:  

Figurative Reasoning: dummy figurative reasoning for:  

OCR Text: when you hear something go bump in
the dark but then you remember you
don't care whether you live or die

 

Figurative Reasoning: dummy figurative reasoning for: when you hear something go bump in
the dark but then you remember you
don't care whether you live or die

 

OCR Text: When u check ur phone after 3 hours

and u have 0 notifications.

 

Figurative Reasoning: dummy figurative reasoning for: When u check ur phone after 3 hours

and u have 0 notifications.

 

OCR Text: Sa MAN A ere e)

JUST A LITTLE TIRED...

Figurative Reasoning: dummy figurative reasoning for: Sa MAN A ere e)

JUST A LITTLE

In [3]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import torch
from sentence_transformers import SentenceTransformer, util
import torch.nn as nn

class RAGModule(nn.Module):
    def __init__(self, model_name='paraphrase-MiniLM-L6-v2'):

        super(RAGModule, self).__init__()
        self.encoder = SentenceTransformer(model_name)

    def encode_text(self, text):

        embedding = self.encoder.encode(text, convert_to_tensor=True)
        return embedding

    def fuse_embeddings(self, ocr_text, reasoning):

        emb_ocr = self.encode_text(ocr_text)          
        emb_reasoning = self.encode_text(reasoning)    
        fused = torch.cat([emb_ocr, emb_reasoning], dim=-1) 
        return fused

    def retrieve_similar(self, query_embedding, database_embeddings, top_k=1):

        cos_scores = util.cos_sim(query_embedding, database_embeddings)  
        top_results = torch.topk(cos_scores, k=top_k)
        return top_results 


if __name__ == "__main__":
    rag_module = RAGModule()

    database_samples = [
        ("when you hear something go bump in the dark",
         "dummy figurative reasoning for: when you hear something go bump in the dark"),
        ("When u check ur phone after 3 hours and u have 0 notifications.",
         "dummy figurative reasoning for: When u check ur phone after 3 hours and u have 0 notifications."),
        ("Sa MAN A ere e) JUST A LITTLE TIRED...",
         "dummy figurative reasoning for: Sa MAN A ere e) JUST A LITTLE TIRED...")
    ]

    database_embeddings_list = []
    for ocr_text, reasoning in database_samples:
        fused_embedding = rag_module.fuse_embeddings(ocr_text, reasoning)
        database_embeddings_list.append(fused_embedding)
    database_embeddings = torch.vstack(database_embeddings_list)  

    query_ocr = "When u check ur phone after 3 hours and see nothing"
    query_reasoning = "dummy figurative reasoning for: When u check ur phone after 3 hours and see nothing"

    query_embedding = rag_module.fuse_embeddings(query_ocr, query_reasoning)
    if query_embedding.dim() == 1:
        query_embedding = query_embedding.unsqueeze(0) 

    top_scores, top_indices = rag_module.retrieve_similar(query_embedding, database_embeddings, top_k=1)

    print("Top similar indices:", top_indices)
    print("Top cosine similarity scores:", top_scores)


Top similar indices: tensor([[1]])
Top cosine similarity scores: tensor([[0.9131]])


In [ ]:
import torch
import torch.nn as nn
from sentence_transformers import SentenceTransformer, util


class RAGModule(nn.Module):
    def __init__(self, model_name='paraphrase-MiniLM-L6-v2'):
   
        super(RAGModule, self).__init__()
        self.encoder = SentenceTransformer(model_name)

    def encode_text(self, text):

        embedding = self.encoder.encode(text, convert_to_tensor=True)
        return embedding

    def fuse_embeddings(self, ocr_text, reasoning):
 
        emb_ocr = self.encode_text(ocr_text)         
        emb_reasoning = self.encode_text(reasoning)   
        fused = torch.cat([emb_ocr, emb_reasoning], dim=-1)  
        return fused

    def retrieve_similar(self, query_embedding, database_embeddings, top_k=1):
    
        cos_scores = util.cos_sim(query_embedding, database_embeddings)  
        top_results = torch.topk(cos_scores, k=top_k)
        return top_results  


class M3HClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        """
        A simple classifier that maps the fused embedding to output logits.
        """
        super(M3HClassifier, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, fused_embedding):
        logits = self.fc(fused_embedding)
        return logits

class M3HModel(nn.Module):
    def __init__(self, rag_module, classifier):
        super(M3HModel, self).__init__()
        self.rag_module = rag_module
        self.classifier = classifier

    def forward(self, ocr_text, reasoning):
        
        fused_embedding = self.rag_module.fuse_embeddings(ocr_text, reasoning)
        if fused_embedding.dim() == 1:
            fused_embedding = fused_embedding.unsqueeze(0)
            
        logits = self.classifier(fused_embedding)
        return logits


if __name__ == "__main__":
    rag_module = RAGModule()

    fused_dim = 768
    num_classes = 6  

    classifier = M3HClassifier(input_dim=fused_dim, num_classes=num_classes)

    model = M3HModel(rag_module, classifier)

    ocr_text = "When u check ur phone after 3 hours and see nothing"
    reasoning = "dummy figurative reasoning for: When u check ur phone after 3 hours and see nothing"

    logits = model(ocr_text, reasoning)
    print("Logits:", logits)


Logits: tensor([[ 0.2047,  0.1273, -0.1380,  0.2827,  0.0309, -0.0304]],
       grad_fn=<AddmmBackward0>)


In [ ]:
import os
import pandas as pd
from PIL import Image
import pytesseract
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.optim as optim
from sentence_transformers import SentenceTransformer, util


class AnxietyDataset(Dataset):
    def __init__(self, csv_file, image_folder, transform=None):
        self.image_folder = image_folder
        self.transform = transform
        
        data = pd.read_csv(csv_file)
        print("CSV columns:", data.columns)

        print("First few sample_ids from CSV:")
        print(data['sample_id'].head())

        data['filename'] = data['sample_id'].apply(lambda x: str(x) + ".jpg" if not str(x).endswith(".jpg") else x)

        data['label'] = pd.factorize(data['meme_anxiety_categories'])[0]
        print("Unique label mapping:", dict(enumerate(pd.factorize(data['meme_anxiety_categories'])[1])))

        data['exists'] = data['filename'].apply(lambda x: os.path.exists(os.path.join(self.image_folder, x)))
        print("Number of existing files found:", data['exists'].sum())

        self.data = data[data['exists']].reset_index(drop=True)
        print(f"Filtered dataset length: {len(self.data)} (only rows with existing images)")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image_name = row['filename']
        label = row['label']
        image_path = os.path.join(self.image_folder, image_name)

        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        ocr_text = pytesseract.image_to_string(Image.open(image_path).convert("RGB"))
        return ocr_text, label


class FigurativeReasoningModule(nn.Module):
    def __init__(self):
        super(FigurativeReasoningModule, self).__init__()

    def forward(self, ocr_text):
        reasoning = "dummy figurative reasoning for: " + ocr_text
        return reasoning


class RAGModule(nn.Module):
    def __init__(self, model_name='paraphrase-MiniLM-L6-v2'):
        super(RAGModule, self).__init__()
        self.encoder = SentenceTransformer(model_name)

    def encode_text(self, text):
        embedding = self.encoder.encode(text, convert_to_tensor=True)
        return embedding

    def fuse_embeddings(self, ocr_text, reasoning):
        emb_ocr = self.encode_text(ocr_text)
        emb_reasoning = self.encode_text(reasoning)
        fused = torch.cat([emb_ocr, emb_reasoning], dim=-1)
        return fused

    def retrieve_similar(self, query_embedding, database_embeddings, top_k=1):
        cos_scores = util.cos_sim(query_embedding, database_embeddings)
        top_results = torch.topk(cos_scores, k=top_k)
        return top_results

class M3HClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(M3HClassifier, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, fused_embedding):
        logits = self.fc(fused_embedding)
        return logits


class M3HModel(nn.Module):
    def __init__(self, rag_module, classifier):
        super(M3HModel, self).__init__()
        self.rag_module = rag_module
        self.classifier = classifier

    def forward(self, ocr_text, reasoning):
        fused_embedding = self.rag_module.fuse_embeddings(ocr_text, reasoning)
        if fused_embedding.dim() == 1:
            fused_embedding = fused_embedding.unsqueeze(0)
        logits = self.classifier(fused_embedding)
        return logits

# Training Loop (One Epoch)
if __name__ == "__main__":
    rag_module = RAGModule()
    figurative_module = FigurativeReasoningModule()
    fused_dim = 768  
    num_classes = 7  
    classifier = M3HClassifier(input_dim=fused_dim, num_classes=num_classes)
    model = M3HModel(rag_module, classifier)

    optimizer = optim.Adam(model.parameters(), lr=5e-5)
    criterion = nn.CrossEntropyLoss()
    
    csv_train = '/content/Anxiety_Data/anxiety_train.csv'
    image_folder = '/content/Anxiety_Data/anxiety_train_image'
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
    dataset = AnxietyDataset(csv_file=csv_train, image_folder=image_folder, transform=transform)
    dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

    model.train()
    total_loss = 0.0
    for batch in dataloader:
        ocr_texts, labels = batch  
        
        reasoning_batch = [figurative_module(text) for text in ocr_texts]

        logits = model(ocr_texts, reasoning_batch)  

        labels = torch.tensor(labels, dtype=torch.long)

        loss = criterion(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        print("Batch loss:", loss.item())

    avg_loss = total_loss / len(dataloader)
    print("Average loss for epoch:", avg_loss)



CSV columns: Index(['sample_id', 'meme_anxiety_categories'], dtype='object')
First few sample_ids from CSV:
0     TR-318
1    TR-1732
2    TR-1417
3    TR-1582
4     TR-649
Name: sample_id, dtype: object
Unique label mapping: {0: 'Restlessness', 1: 'Lack of Worry Control', 2: 'Nervousness', 3: 'Impending Doom', 4: 'Excessive Worry', 5: 'Difficulty Relaxing', 6: 'Irritatbily'}
Number of existing files found: 2605
Filtered dataset length: 2605 (only rows with existing images)


<ipython-input-21-7db9c7342aa9>:164: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long)


Batch loss: 1.9019361734390259


<ipython-input-21-7db9c7342aa9>:164: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long)


Batch loss: 1.9473706483840942


...

<ipython-input-21-7db9c7342aa9>:164: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long)


Batch loss: 1.7922773361206055


<ipython-input-21-7db9c7342aa9>:164: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long)


Batch loss: 1.8313384056091309
Batch loss: 1.877705693244934
Average loss for epoch: 1.9078971034178704


<ipython-input-21-7db9c7342aa9>:164: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long)


In [ ]:
from sklearn.metrics import f1_score, accuracy_score

model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for batch in dataloader: 
        ocr_texts, labels = batch
        reasoning_batch = [figurative_module(text) for text in ocr_texts]

        logits = model(ocr_texts, reasoning_batch)
        preds = torch.argmax(logits, dim=-1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

macro_f1 = f1_score(all_labels, all_preds, average='macro')
weighted_f1 = f1_score(all_labels, all_preds, average='weighted')
accuracy = accuracy_score(all_labels, all_preds)

print("Evaluation Metrics:")
print("Accuracy:", accuracy)
print("Macro-F1:", macro_f1)
print("Weighted-F1:", weighted_f1)


Evaluation Metrics:
Accuracy: 0.2959692898272553
Macro-F1: 0.23362825820405256
Weighted-F1: 0.26328460430956235


In [ ]:
import os
import pandas as pd
from PIL import Image
import pytesseract
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.optim as optim
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics import accuracy_score, f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


class AnxietyDataset(Dataset):
    def __init__(self, csv_file, image_folder, transform=None):
        self.image_folder = image_folder
        self.transform = transform

        data = pd.read_csv(csv_file)
        print("CSV columns:", data.columns)

        print("First few sample_ids from CSV:")
        print(data['sample_id'].head())

        data['filename'] = data['sample_id'].apply(lambda x: str(x) + ".jpg" if not str(x).endswith(".jpg") else x)

        data['label'] = pd.factorize(data['meme_anxiety_categories'])[0]
        print("Unique label mapping:", dict(enumerate(pd.factorize(data['meme_anxiety_categories'])[1])))

        data['exists'] = data['filename'].apply(lambda x: os.path.exists(os.path.join(self.image_folder, x)))
        print("Number of existing files found:", data['exists'].sum())

        self.data = data[data['exists']].reset_index(drop=True)
        print(f"Filtered dataset length: {len(self.data)} (only rows with existing images)")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image_name = row['filename']
        label = row['label']
        image_path = os.path.join(self.image_folder, image_name)

        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        ocr_text = pytesseract.image_to_string(Image.open(image_path).convert("RGB"))
        return ocr_text, label


class FigurativeReasoningModule(nn.Module):
    def __init__(self):
        super(FigurativeReasoningModule, self).__init__()

    def forward(self, ocr_text):
        reasoning = "dummy figurative reasoning for: " + ocr_text
        return reasoning


class RAGModule(nn.Module):
    def __init__(self, model_name='paraphrase-MiniLM-L6-v2'):
        super(RAGModule, self).__init__()
        self.encoder = SentenceTransformer(model_name)

    def encode_text(self, text):
        embedding = self.encoder.encode(text, convert_to_tensor=True)
        return embedding

    def fuse_embeddings(self, ocr_text, reasoning):
        emb_ocr = self.encode_text(ocr_text)
        emb_reasoning = self.encode_text(reasoning)
        fused = torch.cat([emb_ocr, emb_reasoning], dim=-1)
        return fused.to(device)

    def retrieve_similar(self, query_embedding, database_embeddings, top_k=1):
        cos_scores = util.cos_sim(query_embedding, database_embeddings)
        top_results = torch.topk(cos_scores, k=top_k)
        return top_results


class M3HClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(M3HClassifier, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, fused_embedding):
        logits = self.fc(fused_embedding)
        return logits

class M3HModel(nn.Module):
    def __init__(self, rag_module, classifier):
        super(M3HModel, self).__init__()
        self.rag_module = rag_module
        self.classifier = classifier

    def forward(self, ocr_text, reasoning):
        fused_embedding = self.rag_module.fuse_embeddings(ocr_text, reasoning)
        if fused_embedding.dim() == 1:
            fused_embedding = fused_embedding.unsqueeze(0)
        logits = self.classifier(fused_embedding)
        return logits


if __name__ == "__main__":
    rag_module = RAGModule()
    figurative_module = FigurativeReasoningModule()
    fused_dim = 768  
    num_classes = 7  
    classifier = M3HClassifier(input_dim=fused_dim, num_classes=num_classes)
    model = M3HModel(rag_module, classifier)
    model.to(device)  

    optimizer = optim.Adam(model.parameters(), lr=5e-5)
    criterion = nn.CrossEntropyLoss()

    csv_train = '/content/Anxiety_Data/anxiety_train.csv'
    image_folder = '/content/Anxiety_Data/anxiety_train_image'
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
    dataset = AnxietyDataset(csv_file=csv_train, image_folder=image_folder, transform=transform)
    dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

    num_epochs = 5
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        for batch in dataloader:
            ocr_texts, labels = batch  

            reasoning_batch = [figurative_module(text) for text in ocr_texts]

            logits = model(ocr_texts, reasoning_batch) 

            labels = torch.tensor(labels, dtype=torch.long).to(device)

            loss = criterion(logits, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            print(f"Epoch {epoch+1} Batch loss: {loss.item()}")

        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch+1} Average loss: {avg_loss}")


Using device: cuda
CSV columns: Index(['sample_id', 'meme_anxiety_categories'], dtype='object')
First few sample_ids from CSV:
0     TR-318
1    TR-1732
2    TR-1417
3    TR-1582
4     TR-649
Name: sample_id, dtype: object
Unique label mapping: {0: 'Restlessness', 1: 'Lack of Worry Control', 2: 'Nervousness', 3: 'Impending Doom', 4: 'Excessive Worry', 5: 'Difficulty Relaxing', 6: 'Irritatbily'}
Number of existing files found: 2605
Filtered dataset length: 2605 (only rows with existing images)


<ipython-input-8-37b374e638f2>:175: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long).to(device)


Epoch 1 Batch loss: 1.985504388809204


<ipython-input-8-37b374e638f2>:175: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long).to(device)


Epoch 1 Batch loss: 2.0814919471740723


...

<ipython-input-8-37b374e638f2>:175: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long).to(device)


Epoch 5 Batch loss: 2.2063519954681396


<ipython-input-8-37b374e638f2>:175: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long).to(device)


Epoch 5 Batch loss: 1.68229341506958
Epoch 5 Batch loss: 2.034149646759033
Epoch 5 Batch loss: 1.9502322673797607
Epoch 5 Average loss: 1.6913689268878633


<ipython-input-8-37b374e638f2>:175: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long).to(device)
<ipython-input-8-37b374e638f2>:175: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long).to(device)


In [ ]:
# Evaluation
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for batch in dataloader: 
        ocr_texts, labels = batch
        reasoning_batch = [figurative_module(text) for text in ocr_texts]
        logits = model(ocr_texts, reasoning_batch)
        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
macro_f1 = f1_score(all_labels, all_preds, average='macro')
weighted_f1 = f1_score(all_labels, all_preds, average='weighted')

print("Evaluation Metrics:")
print("Accuracy:", accuracy)
print("Macro-F1:", macro_f1)
print("Weighted-F1:", weighted_f1)


Evaluation Metrics:
Accuracy: 0.4130518234165067
Macro-F1: 0.354402327419053
Weighted-F1: 0.3919029289257237
